# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.9.1'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [6]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [11]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [12]:
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [8]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [9]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X[:, 2] = le.fit_transform(X[:, 2])

In [15]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')

X = np.array(ct.fit_transform(X))

In [17]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [19]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [21]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [22]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [23]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

#### We use SIGMOID AF for outerlayer of Binary Classification problem
#### We use SOFTMAX AF for outerlayer of Multiclass Classification problem
#### We use LINEAR AF for outerlayer of Regression problem

In [24]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [25]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [26]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.5645 - accuracy: 0.7909
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4929 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4667 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4476 - accuracy: 0.7969
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4354 - accuracy: 0.8001
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4273 - accuracy: 0.8039
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4211 - accuracy: 0.8052
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4164 - accuracy: 0.8114
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4092 - accuracy: 0.8232
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3998 - accura

250/250 [==============================] - 1s 3ms/step - loss: 0.3329 - accuracy: 0.8646
Epoch 82/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3328 - accuracy: 0.8634
Epoch 83/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3325 - accuracy: 0.8635
Epoch 84/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3316 - accuracy: 0.8631
Epoch 85/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3325 - accuracy: 0.8627
Epoch 86/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3322 - accuracy: 0.8641
Epoch 87/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3312 - accuracy: 0.8631
Epoch 88/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3319 - accuracy: 0.8646
Epoch 89/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3320 - accuracy: 0.8634
Epoch 90/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3316 - accuracy: 

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

To use our ANN model to predict if the customer with the following informations will leave the bank: 

1 - Exited
0 - Not exited

- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40 years old
- Tenure: 3 years
- Balance: USD 60000
- Number of Products: 2
- Does this customer have a credit card ? Yes
- Is this customer an Active Member: Yes
- Estimated Salary: USD 50000

**Solution**

In [27]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 219ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

### Predicting the Test set results

In [28]:
y_pred = ann.predict(X_test)

63/63 [==============================] - 0s 2ms/step


In [29]:
y_pred = (y_pred > 0.5)

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(f'The Confusion matrix is as follows {cm}')

acc = accuracy_score(y_test, y_pred)
print(f'The Accuracy is as follows {acc}')


The Confusion matrix is as follows [[1505   90]
 [ 189  216]]
The Accuracy is as follows 0.8605
